In [1]:
!pip install networkx

# Graph Walk
引数:<br>
<b>N</b><br>
タイプ:int,デフォルト:100:生成するタグセットの数<br>
<b>L</b><br>
タイプ:int,デフォルト:2:1つのタグセットにつき生成する最大のタグ数<br>
<b>node_selection</b><br>
タイプ:int,デフォルト:1:0の場合、タグセットの最初のノードを次数に従う確率で選択する。1の場合、優先的選択に従う(初期状態は次数)、2の場合、ランダムに選択する。<br>
<b>edge_selection</b><br>
タイプ:int,デフォルト:1:0の場合、次のノードを隣り合うノードからランダムに選択する。1の場合、優先的選択に従う(初期状態は次数)、2の場合、初期状態で固定する。<br>
<b>initial_edge</b><br>
タイプ:NoneType or nested list of int,デフォルト:None:初期の隣接行列。もしNoneの場合、初期の隣接行列はinitial_Graphに従う。<br>
<b>initial_Graph</b><br>
タイプ:nx.classes.graph.Graph,デフォルト:nx.relaxed_caveman_graph(100,5,0.1):initital_edgeがNoneの場合、こちらに従って、隣接行列を生成する。<br>
<b>initial_Graph</b><br>
タイプ:nx.classes.graph.Graph,デフォルト:nx.relaxed_caveman_graph(100,5,0.1):initital_edgeがNoneの場合、こちらに従って、隣接行列を生成する。<br>
<b>fixed_L</b><br>
タイプ:bool,デフォルト:False:Trueの場合、それぞれのタイムステップごとにL分だけdistinctなタグを生成する。もし孤立しているノード群がある場合無限ループになる可能性がある。<br>
<b>おまけ</b>
nx.relaxed_caveman_graph(c,s,p)はクリーク数cとクリークサイズsからグラフを生成するcaveman_graphにたいして、新規edgeがpの確率で別のノードと繋げて生成する。


In [90]:

class Graph_Walk:
    import random
    import networkx as nx
    def VE(self,st1,X,st2):
        raise ValueError(st1+" should be "+str(st2)+"! "+st1+" = "+str(X))
    def TC(self,s,X,types):#type check
        if type(X) not in types:
            self.VE(s,X,types)
    def TC_list(self,s,ls,types):#type check nest
        for l in ls:
            if type(l) not in types:
                self.VE("Contetnts of "+s,ls,types)
    def TC_nested_list(self,s,ls,types):#type check nest
        for l in ls:
            self.TC_list(s,ls,types)
    def equ_N(self,st,ls):#check equal N
        if self.N==len(ls):
            self.VE("Length of "+st,len(ls),"N")
    def equ_N_nest(self,st,ls):#check equal N to all element ls.
        for i,l in enumerate(ls):
            self.equ_N("Length of "+st+"["+str(i)+"]",l)
    

    def setN(self,N):
        self.TC("N",N,[int])
        self.N=N
    def setL(self,L):
        self.TC("L",L,[int,list])
        if type(L) == list:
            self.TC_list("L",L,[int])
            self.equ_N("L",L)
            self.equ_N_nest("L",L)
            self.TC_nested_list("L",L,[int])
            self.L=L
        else:
            self.L=[L]*self.N
    def setNode_selection(self,NS):
        #0=Edge of randomly selected edge
        #1=PA
        #2=initilal PA
        self.TC("node_selection",NS,[int])
        if NS>=3:
            self.VE("node_selection",NS,"2 or less")
        self.NS=NS
    def setEdge_selection(self,ES):
        #0=RW
        #1=PA
        #2=initilal PA
        self.TC("edge_selection",ES,[int])
        if ES>=3:
            self.VE("edge_selection",ES,"2 or less")
        self.ES=ES
        
    def setInitilal_edge(self,IE):
        #nested list of int
        self.TC("initilal_edge",IE,[list,type(None)])
        if type(IE)==list:
            self.TC_list("initilal_edge",IE,[list])
            self.TC_nested_list("initilal_edge",IE,[int])
        else:
            self.edges=IE
            
    def setInitial_Graph(self,G):
        self.TC("initial_graph",G,[nx.classes.graph.Graph])
        self.graph=G
        self.nodes=G.nodes
        self.edges=nx.adjacency_matrix(G).toarray()
    def setFixedL(self,fixedL):
        self.TC("fixed_L",fixedL,[bool])
        self.fixed_L=fixedL
        
    def setNodeVase(self):
        edges=self.edges
        self.NodeVase=[i for i in range(self.N) for j in range(self.N) for k in range(edges[i][j])]
    def setEdgeVase(self):
        edges=self.edges
        self.EdgeVase=[(i,j) for i in range(self.N) for j in range(i,self.N) if edges[i][j]!=0]
        
        
    def __init__(self,N=100,L=2,node_selection=1,edge_selection=1,initial_edge=None,initial_Graph=nx.relaxed_caveman_graph(100,5,0.1),fixed_L=False):
        self.setN(N)
        self.setL(L)
        self.setNode_selection(node_selection)#0=Edge,#1=PA,#2=initialPA
        self.setEdge_selection(edge_selection)#0=RW,#1=PA,#2=initialPA
        self.Graph_Flag=0
        
        if initial_edge==None:
            self.setInitial_Graph(initial_Graph)
        else:
            self.setInitilal_edge(initial_edge)
        self.setFixedL(fixed_L)
        self.setNodeVase()
        self.setEdgeVase()
        self.generate()
    def PA(self,ls):#
        return self.random.choice(ls)
    def ChoiceNode(self):
        edges=self.edges
        
        if self.NS==0:#Edge
            vase=self.EdgeVase
            edge=self.PA(vase)
            node=self.PA(edge)#PAではないが、実質同じ処理なため、利用。
        elif self.NS==1:#PA
            vase=self.NodeVase
            node=self.PA(vase)
            self.NodeVase.append(node)
        else:
            node=self.random.randint(0,len(edges))
            
        return node
    def RW(self,source):
        edges=self.edges[source]#nodeから伸びているedgeのリストを取得。0なら繋がっていない。
        index_connected_edge=[i for i in range(len(edges)) if edges[i]!=0]
        if self.ES==0:#RW
            node=self.PA(index_connected_edge)
        else:
            connected_edge_vase=[i for i in index_connected_edge for _ in range(edges[i])]
            node=self.PA(connected_edge_vase)
        return node
    def get_tagset(self,i):
        tagset=[self.ChoiceNode()]
        for _ in range(self.L[i]-1):
            node=self.RW(tagset[-1])
            
            while node in tagset and self.fixed_L:
                node=self.RW(tagset[-1])
            tagset.append(node)
            if self.NS==1:
                self.NodeVase.append(tagset[-1])
        if self.ES==1:
            for t1,t2 in zip(tagset,tagset[1:]):
                self.edges[t1][t2]+=1
                self.edges[t2][t1]+=1
                    
        return list(set(tagset))
        
        
    def generate(self):
        tagsets=[]
        for i in range(self.N):
            tagsets.append(self.get_tagset(i))
        self.result=tagsets
    def getResult(self):
        return self.result
    def getNodeVase(self):
        return self.NodeVase
    def getEdge(self):
        return self.edges
        
        
        
    
    


In [94]:
GW=Graph_Walk()
GW.getResult()

[[96, 35],
 [27, 28],
 [40, 43],
 [6, 383],
 [8, 7],
 [34, 30],
 [88, 87],
 [0, 1],
 [80, 83],
 [1, 2],
 [83, 84],
 [26, 28],
 [89, 86],
 [89, 87],
 [8, 7],
 [58, 59],
 [98, 356],
 [75, 78],
 [91, 92],
 [46, 47],
 [315, 21],
 [89, 86],
 [75, 79],
 [13, 14],
 [96, 35],
 [80, 82],
 [91, 93],
 [5, 6],
 [75, 79],
 [10, 14],
 [80, 83],
 [0, 3],
 [41, 213],
 [49, 47],
 [19, 15],
 [96, 98],
 [48, 46],
 [73, 71],
 [10, 11],
 [315, 318],
 [13, 14],
 [96, 99],
 [40, 43],
 [11, 191],
 [8, 7],
 [20, 23],
 [8, 7],
 [40, 44],
 [21, 22],
 [46, 47],
 [295, 15],
 [89, 86],
 [91, 93],
 [48, 46],
 [58, 59],
 [21, 22],
 [6, 383],
 [36, 37],
 [24, 22],
 [96, 99],
 [20, 22],
 [72, 74],
 [10, 13],
 [80, 84],
 [0, 1],
 [91, 94],
 [91, 92],
 [60, 61],
 [10, 13],
 [89, 85],
 [33, 31],
 [40, 43],
 [76, 79],
 [88, 455],
 [2, 3],
 [40, 43],
 [50, 51],
 [296, 82],
 [6, 383],
 [38, 39],
 [60, 61],
 [8, 11],
 [9, 5],
 [17, 18],
 [37, 39],
 [96, 35],
 [96, 99],
 [11, 191],
 [45, 47],
 [50, 54],
 [21, 22],
 [73, 71],
 

In [95]:
GW=Graph_Walk(L=3,fixed_L=True)
GW.getResult()

[[85, 86, 87],
 [45, 46, 47],
 [329, 52, 53],
 [10, 12, 13],
 [0, 1, 2],
 [72, 70, 79],
 [70, 78, 79],
 [81, 82, 83],
 [40, 296, 43],
 [72, 73, 74],
 [9, 5, 7],
 [65, 67, 69],
 [90, 92, 94],
 [10, 11, 14],
 [26, 27, 28],
 [36, 37, 39],
 [35, 37, 38],
 [11, 14, 191],
 [33, 34, 30],
 [16, 17, 19],
 [20, 22, 23],
 [20, 21, 22],
 [48, 45, 46],
 [33, 125, 30],
 [57, 59, 55],
 [72, 70, 79],
 [16, 19, 15],
 [10, 13, 14],
 [20, 22, 23],
 [16, 17, 18],
 [65, 67, 69],
 [36, 37, 39],
 [10, 12, 13],
 [66, 67, 69],
 [26, 27, 28],
 [91, 93, 94],
 [92, 93, 94],
 [90, 92, 94],
 [9, 5, 7],
 [17, 18, 19],
 [76, 78, 79],
 [24, 20, 22],
 [11, 12, 13],
 [296, 82, 40],
 [25, 26, 27],
 [36, 38, 39],
 [35, 36, 39],
 [66, 67, 69],
 [96, 97, 98],
 [16, 18, 336],
 [36, 37, 38],
 [64, 60, 62],
 [65, 67, 69],
 [88, 85, 86],
 [90, 92, 94],
 [16, 18, 19],
 [56, 58, 59],
 [65, 66, 68],
 [24, 21, 22],
 [88, 85, 86],
 [91, 92, 93],
 [35, 37, 38],
 [10, 12, 14],
 [32, 30, 31],
 [81, 83, 84],
 [65, 67, 69],
 [297, 295, 1